## 5-1 如何使用Keras 打造神經網路

** Keras 預建神經層類型 **

* 多層感知器(MLP) : 之前都是在說這個
* 卷積神經網路(CNN) : 做影像識別很準
* 循環神經網路(RNN) : 做NLP



In [6]:
# 5-2-1 認識皮馬印地安人的糖尿病資料 

import pandas as pd

np.random.seed(10) # 指定亂數種子
# 載入糖尿病資料集
df = pd.read_csv("./diabetes.csv")

print(df.head())
print(df.shape)

   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   
2            8      183             64              0        0  23.3   
3            1       89             66             23       94  28.1   
4            0      137             40             35      168  43.1   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0  
2                     0.672   32        1  
3                     0.167   21        0  
4                     2.288   33        1  
(768, 9)


In [15]:
# 5-2-1 認識皮馬印地安人的糖尿病資料 
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense

np.random.seed(10) # 指定亂數種子
# 載入糖尿病資料集
df = pd.read_csv("./diabetes.csv")

print(df.head())
print(df.shape)

dataset = df.values
np.random.shuffle(dataset) # 使用亂數打亂資料 

# 分割成特徵資料與標籤資料
X = dataset[:,0:8]
Y = dataset[:,8]

# 特徵標準化
X -= X.mean(axis=0)
X /= X.std(axis=0)


# 定義模型
model = Sequential()
model.add(Dense(10,input_shape=(8,), activation="relu"))
model.add(Dense(8, activation="relu"))
model.add(Dense(1, activation="sigmoid"))
model.summary() # 顯示模型摘要資訊 

# 編譯模型
model.compile(loss="binary_crossentropy", optimizer="sgd", metrics=["accuracy"])

# 訓練模型
model.fit(X,Y, epochs=150, batch_size=10)

# 評估模型
loss,accuracy = model.evaluate(X,Y)
print("準確度 = {:.2f}".format(accuracy))

   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   
2            8      183             64              0        0  23.3   
3            1       89             66             23       94  28.1   
4            0      137             40             35      168  43.1   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0  
2                     0.672   32        1  
3                     0.167   21        0  
4                     2.288   33        1  
(768, 9)
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 10)                90        
_________________________________________________________________
dense_14

768/768 [==============================] - 0s 171us/step - loss: 0.4583 - acc: 0.7747
Epoch 70/150
768/768 [==============================] - 0s 154us/step - loss: 0.4577 - acc: 0.7734
Epoch 71/150
768/768 [==============================] - 0s 160us/step - loss: 0.4573 - acc: 0.7773
Epoch 72/150
768/768 [==============================] - 0s 156us/step - loss: 0.4569 - acc: 0.7734
Epoch 73/150
768/768 [==============================] - 0s 150us/step - loss: 0.4567 - acc: 0.7786
Epoch 74/150
768/768 [==============================] - 0s 163us/step - loss: 0.4565 - acc: 0.7747
Epoch 75/150
768/768 [==============================] - 0s 157us/step - loss: 0.4560 - acc: 0.7773
Epoch 76/150
768/768 [==============================] - 0s 156us/step - loss: 0.4557 - acc: 0.7760
Epoch 77/150
768/768 [==============================] - 0s 154us/step - loss: 0.4553 - acc: 0.7812
Epoch 78/150
768/768 [==============================] - 0s 157us/step - loss: 0.4553 - acc: 0.7799
Epoch 79/150
768/768 [=

768/768 [==============================] - 0s 209us/step
準確度 = 0.80


**Dense** 

全連結層,在Keran的架構叫 Dense

In [18]:
# 5-2-1 認識皮馬印地安人的糖尿病資料 
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

np.random.seed(10) # 指定亂數種子
# 載入糖尿病資料集
df = pd.read_csv("./diabetes.csv")

print(df.head())
print(df.shape)

dataset = df.values
np.random.shuffle(dataset) # 使用亂數打亂資料 

# 分割成特徵資料與標籤資料
X = dataset[:,0:8]
Y = dataset[:,8]

# 特徵標準化
X -= X.mean(axis=0)
X /= X.std(axis=0)
# One-hot encoding
Y = to_categorical(Y)

# 定義模型
model = Sequential()
model.add(Dense(10,input_shape=(8,), activation="relu"))
model.add(Dense(8, activation="relu"))
model.add(Dense(2, activation="softmax"))
model.summary() # 顯示模型摘要資訊 

# 編譯模型
model.compile(loss="binary_crossentropy", optimizer="sgd", metrics=["accuracy"])

# 訓練模型
model.fit(X,Y, epochs=150, batch_size=10)

# 評估模型
loss,accuracy = model.evaluate(X,Y)
print("準確度 = {:.2f}".format(accuracy))

   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   
2            8      183             64              0        0  23.3   
3            1       89             66             23       94  28.1   
4            0      137             40             35      168  43.1   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0  
2                     0.672   32        1  
3                     0.167   21        0  
4                     2.288   33        1  
(768, 9)
Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 10)                90        
_________________________________________________________________
dense_23

768/768 [==============================] - 0s 223us/step - loss: 0.4343 - acc: 0.7826
Epoch 70/150
768/768 [==============================] - 0s 173us/step - loss: 0.4333 - acc: 0.7786
Epoch 71/150
768/768 [==============================] - 0s 187us/step - loss: 0.4330 - acc: 0.7839
Epoch 72/150
768/768 [==============================] - 0s 181us/step - loss: 0.4326 - acc: 0.7812
Epoch 73/150
768/768 [==============================] - 0s 192us/step - loss: 0.4325 - acc: 0.7812
Epoch 74/150
768/768 [==============================] - 0s 188us/step - loss: 0.4325 - acc: 0.7878
Epoch 75/150
768/768 [==============================] - 0s 184us/step - loss: 0.4320 - acc: 0.7839
Epoch 76/150
768/768 [==============================] - 0s 183us/step - loss: 0.4315 - acc: 0.7839
Epoch 77/150
768/768 [==============================] - 0s 198us/step - loss: 0.4314 - acc: 0.7839
Epoch 78/150
768/768 [==============================] - 0s 182us/step - loss: 0.4310 - acc: 0.7852
Epoch 79/150
768/768 [=

768/768 [==============================] - 0s 170us/step
準確度 = 0.79


In [19]:
# 5-2-1 認識皮馬印地安人的糖尿病資料 
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

np.random.seed(10) # 指定亂數種子
# 載入糖尿病資料集
df = pd.read_csv("./diabetes.csv")

print(df.head())
print(df.shape)

dataset = df.values
np.random.shuffle(dataset) # 使用亂數打亂資料 

# 分割成特徵資料與標籤資料
X = dataset[:,0:8]
Y = dataset[:,8]

# 特徵標準化
X -= X.mean(axis=0)
X /= X.std(axis=0)
# One-hot encoding
Y = to_categorical(Y)

# 定義模型
model = Sequential()
model.add(Dense(10, input_shape=(8,),
                kernel_initializer="random_uniform", 
                bias_initializer="ones",
                activation="relu"))
model.add(Dense(8, kernel_initializer="random_uniform",
                bias_initializer="ones",
                activation="relu"))
model.add(Dense(2, kernel_initializer="random_uniform",
                bias_initializer="ones",
                activation="softmax"))
#model.summary() # 顯示模型摘要資訊 

# 編譯模型
model.compile(loss="binary_crossentropy", optimizer="sgd", metrics=["accuracy"])

# 訓練模型
model.fit(X,Y, epochs=150, batch_size=10)

# 評估模型
loss,accuracy = model.evaluate(X,Y)
print("準確度 = {:.2f}".format(accuracy))

   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   
2            8      183             64              0        0  23.3   
3            1       89             66             23       94  28.1   
4            0      137             40             35      168  43.1   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0  
2                     0.672   32        1  
3                     0.167   21        0  
4                     2.288   33        1  
(768, 9)
Epoch 1/150
768/768 [==============================] - 1s 884us/step - loss: 0.6597 - acc: 0.6302
Epoch 2/150
768/768 [==============================] - 0s 186us/step - loss: 0.6486 - acc: 0.6510
Epoch 3/150
768/768 [==============================] - 0s 171us/step - loss: 0.6480 - acc: 0.6510


768/768 [==============================] - 0s 195us/step - loss: 0.4684 - acc: 0.7786
Epoch 77/150
768/768 [==============================] - 0s 179us/step - loss: 0.4672 - acc: 0.7826
Epoch 78/150
768/768 [==============================] - 0s 231us/step - loss: 0.4669 - acc: 0.7799
Epoch 79/150
768/768 [==============================] - 0s 218us/step - loss: 0.4660 - acc: 0.7773
Epoch 80/150
768/768 [==============================] - 0s 194us/step - loss: 0.4656 - acc: 0.7760
Epoch 81/150
768/768 [==============================] - 0s 264us/step - loss: 0.4657 - acc: 0.7786
Epoch 82/150
768/768 [==============================] - 0s 297us/step - loss: 0.4658 - acc: 0.7852
Epoch 83/150
768/768 [==============================] - 0s 214us/step - loss: 0.4670 - acc: 0.7786
Epoch 84/150
768/768 [==============================] - 0s 269us/step - loss: 0.4668 - acc: 0.7826
Epoch 85/150
768/768 [==============================] - 0s 211us/step - loss: 0.4657 - acc: 0.7812
Epoch 86/150
768/768 [=

In [20]:
# 5-2-1 認識皮馬印地安人的糖尿病資料 
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

np.random.seed(10) # 指定亂數種子
# 載入糖尿病資料集
df = pd.read_csv("./diabetes.csv")

print(df.head())
print(df.shape)

dataset = df.values
np.random.shuffle(dataset) # 使用亂數打亂資料 

# 分割成特徵資料與標籤資料
X = dataset[:,0:8]
Y = dataset[:,8]

# 特徵標準化
X -= X.mean(axis=0)
X /= X.std(axis=0)
# One-hot encoding
Y = to_categorical(Y)

# 定義模型
model = Sequential()
model.add(Dense(10, input_shape=(8,),
                kernel_initializer="random_uniform", 
                bias_initializer="ones",
                activation="relu"))
model.add(Dense(8, kernel_initializer="random_uniform",
                bias_initializer="ones",
                activation="relu"))
model.add(Dense(2, kernel_initializer="random_uniform",
                bias_initializer="ones",
                activation="softmax"))
#model.summary() # 顯示模型摘要資訊 

# 編譯模型
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# 訓練模型
model.fit(X,Y, epochs=150, batch_size=10)

# 評估模型
loss,accuracy = model.evaluate(X,Y)
print("準確度 = {:.2f}".format(accuracy))

   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   
2            8      183             64              0        0  23.3   
3            1       89             66             23       94  28.1   
4            0      137             40             35      168  43.1   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0  
2                     0.672   32        1  
3                     0.167   21        0  
4                     2.288   33        1  
(768, 9)
Epoch 1/150
768/768 [==============================] - 1s 924us/step - loss: 0.6621 - acc: 0.6146
Epoch 2/150
768/768 [==============================] - 0s 227us/step - loss: 0.6337 - acc: 0.6510
Epoch 3/150
768/768 [==============================] - 0s 202us/step - loss: 0.5983 - acc: 0.6510


768/768 [==============================] - 0s 332us/step - loss: 0.4531 - acc: 0.7826
Epoch 77/150
768/768 [==============================] - 0s 260us/step - loss: 0.4511 - acc: 0.7865
Epoch 78/150
768/768 [==============================] - 0s 300us/step - loss: 0.4528 - acc: 0.7826
Epoch 79/150
768/768 [==============================] - 0s 250us/step - loss: 0.4529 - acc: 0.7799
Epoch 80/150
768/768 [==============================] - 0s 226us/step - loss: 0.4500 - acc: 0.7826
Epoch 81/150
768/768 [==============================] - 0s 308us/step - loss: 0.4512 - acc: 0.7799
Epoch 82/150
768/768 [==============================] - 0s 233us/step - loss: 0.4491 - acc: 0.7799
Epoch 83/150
768/768 [==============================] - 0s 218us/step - loss: 0.4484 - acc: 0.7812
Epoch 84/150
768/768 [==============================] - 0s 427us/step - loss: 0.4489 - acc: 0.7812
Epoch 85/150
768/768 [==============================] - 0s 214us/step - loss: 0.4484 - acc: 0.7826
Epoch 86/150
768/768 [=